In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq fastembed==0.3.4 --progress-bar off
!pip install -qqq sqlite-vec==0.1.1 --progress-bar off
!pip install -qqq groq==0.9.0 --progress-bar off

  Preparing metadata (setup.py) ... done


In [2]:
import sqlite3
from textwrap import dedent
from typing import List

import sqlite_vec
from fastembed import TextEmbedding
from google.colab import userdata
from groq import Groq
from groq.types.chat import ChatCompletionMessage
from sqlite_vec import serialize_float32

client = Groq(api_key=userdata.get("GROQ_API_KEY"))
MODEL = "llama-3.1-70b-versatile"
TEMPERATURE = 0
MAX_TOKENS = 4096

db = sqlite3.connect("faq.sqlite3")
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

In [3]:
%load_ext sql
%sql sqlite:///faq.sqlite3

In [4]:
KNOWLEDGE_BASE = [
    {
        "question": "Are the parts brand new?",
        "answer": "All parts we sell are OEM or aftermarket OEM replacements and are brand new (never used). We guarantee that each part has not been previously installed or used in any vehicle.",
    },
    {
        "question": "What is the return policy?",
        "answer": "You can return any part within 30 days of purchase for a full refund. The part must be in its original packaging and condition. Customized or special-order parts are non-returnable.",
    },
    {
        "question": "Do you offer a warranty on your parts?",
        "answer": "Yes, we offer a minimum one-year warranty on all parts. Some parts may have extended warranties provided by the manufacturer. Please check the specific part’s warranty details for more information.",
    },
    {
        "question": "How long does shipping take?",
        "answer": "Standard shipping within the continental US typically takes 3-5 business days. Expedited shipping options are available at an additional cost. International shipping times vary based on location.",
    },
    {
        "question": "Can I pick up my order in-store?",
        "answer": "Yes, we offer in-store pickup for online orders. You can select the pickup option at checkout and choose your preferred store location. We will notify you when your order is ready for pickup.",
    },
    {
        "question": "Do you offer installation services?",
        "answer": "We do not offer installation services directly. However, we can recommend certified local mechanics and auto repair shops. Many of our locations have partnerships with trusted service providers.",
    },
    {
        "question": "How can I track my order?",
        "answer": "Once your order is shipped, you will receive a tracking number via email. You can use this tracking number on our website or the carrier’s website to monitor the delivery status of your order.",
    },
    {
        "question": "Do you price match competitors?",
        "answer": "Yes, we offer a price match guarantee on identical parts from authorized retailers. Simply provide proof of the lower price, and we will match it. Price matching is subject to verification and certain conditions.",
    },
    {
        "question": "Are the parts compatible with my vehicle?",
        "answer": "We provide detailed compatibility information for each part. You can use our online tool to enter your vehicle’s make, model, and year to ensure the part fits. If you have any doubts, our customer service team is here to help.",
    },
    {
        "question": "Can I cancel my order?",
        "answer": "You can cancel your order if it has not yet been shipped. Please contact our customer service as soon as possible to process the cancellation. If the order has already been shipped, you may need to follow the return process.",
    },
    {
        "question": "Do you sell used or refurbished parts?",
        "answer": "We primarily sell brand new OEM and aftermarket parts. However, we do offer a selection of refurbished parts which are clearly marked and come with their own warranties. All refurbished parts undergo strict quality control checks.",
    },
    {
        "question": "What payment methods do you accept?",
        "answer": "We accept a variety of payment methods including major credit cards (Visa, MasterCard, American Express), PayPal, and Apple Pay. For in-store purchases, we also accept cash and debit cards.",
    },
]

## Add Documents to Database

In [5]:
db.execute(
    """
    CREATE TABLE documents(
        id INTEGER PRIMARY KEY,
        question TEXT,
        answer TEXT
    );
"""
)

In [6]:
with db:
    for i, doc in enumerate(KNOWLEDGE_BASE):
        db.execute(
            "INSERT INTO documents(id, question, answer) VALUES(?, ?, ?)",
            [i, doc["question"], doc["answer"]],
        )

## Document Embeddings

In [7]:
embedding_model = TextEmbedding()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

In [8]:
document_embeddings = list(embedding_model.embed(KNOWLEDGE_BASE[0]["question"]))
len(document_embeddings), document_embeddings[0].shape

(1, (384,))

In [9]:
db.execute(
    f"""
        CREATE VIRTUAL TABLE document_embeddings USING vec0(
          id INTEGER PRIMARY KEY,
          embedding FLOAT[{document_embeddings[0].shape[0]}]
        );
    """
)

In [10]:
with db:
    document_rows = db.execute("SELECT id, question, answer FROM documents").fetchall()
    documents = [f"{row[1]}\n{row[2]}" for row in document_rows]
    print(documents[0])
    document_embeddings = list(embedding_model.embed(documents))
    for (id, _, _), embedding in zip(document_rows, document_embeddings):
        db.execute(
            "INSERT INTO document_embeddings(id, embedding) VALUES (?, ?)",
            [id, serialize_float32(embedding)],
        )

Are the parts brand new?
All parts we sell are OEM or aftermarket OEM replacements and are brand new (never used). We guarantee that each part has not been previously installed or used in any vehicle.


## Database Structure

In [11]:
%%sql
SELECT
    name,
    type,
    sql
FROM
    sqlite_master
WHERE
    type IN ('table', 'index', 'view', 'trigger')
ORDER BY
    type,
    name;

 * sqlite:///faq.sqlite3
Done.


name,type,sql
sqlite_autoindex_document_embeddings_vector_chunks00_1,index,None
document_embeddings,table,"CREATE VIRTUAL TABLE document_embeddings USING vec0( id INTEGER PRIMARY KEY, embedding FLOAT[384] )"
document_embeddings_chunks,table,"CREATE TABLE ""document_embeddings_chunks""(chunk_id INTEGER PRIMARY KEY AUTOINCREMENT,size INTEGER NOT NULL,validity BLOB NOT NULL,rowids BLOB NOT NULL)"
document_embeddings_rowids,table,"CREATE TABLE ""document_embeddings_rowids""(rowid INTEGER PRIMARY KEY AUTOINCREMENT,id,chunk_id INTEGER,chunk_offset INTEGER)"
document_embeddings_vector_chunks00,table,"CREATE TABLE ""document_embeddings_vector_chunks00""(rowid PRIMARY KEY,vectors BLOB NOT NULL)"
documents,table,"CREATE TABLE documents( id INTEGER PRIMARY KEY, question TEXT, answer TEXT )"
sqlite_sequence,table,"CREATE TABLE sqlite_sequence(name,seq)"


## Find Similar Documents

In [12]:
query = "What is the warranty?"
query_embeddings = list(embedding_model.embed([query]))
query_embedding = query_embeddings[0]

In [13]:
results = db.execute(
    """
      SELECT
        document_embeddings.id,
        distance,
        question,
        answer
      FROM document_embeddings
      LEFT JOIN documents ON documents.id = document_embeddings.id
      WHERE embedding MATCH ?
        AND k = 3
      ORDER BY distance
    """,
    [serialize_float32(query_embedding)],
).fetchall()

In [14]:
results

[(2,
  0.7537915110588074,
  'Do you offer a warranty on your parts?',
  'Yes, we offer a minimum one-year warranty on all parts. Some parts may have extended warranties provided by the manufacturer. Please check the specific part’s warranty details for more information.'),
 (1,
  0.8563459515571594,
  'What is the return policy?',
  'You can return any part within 30 days of purchase for a full refund. The part must be in its original packaging and condition. Customized or special-order parts are non-returnable.'),
 (10,
  0.8647741675376892,
  'Do you sell used or refurbished parts?',
  'We primarily sell brand new OEM and aftermarket parts. However, we do offer a selection of refurbished parts which are clearly marked and come with their own warranties. All refurbished parts undergo strict quality control checks.')]

## RAG

In [15]:
def retrieve_context(
    query: str, k: int = 3, embedding_model: TextEmbedding = embedding_model
) -> str:
    query_embedding = list(embedding_model.embed([query]))[0]
    results = db.execute(
        """
        SELECT
            document_embeddings.id,
            distance,
            question,
            answer
        FROM document_embeddings
        LEFT JOIN documents ON documents.id = document_embeddings.id
        WHERE embedding MATCH ?
            AND k = ?
        ORDER BY distance
        """,
        [serialize_float32(query_embedding), k],
    ).fetchall()
    return "\n---\n".join([f"{item[2]}\n{item[3]}" for item in results])

In [16]:
query = "What is the warranty for the parts?"
print(retrieve_context(query))

Do you offer a warranty on your parts?
Yes, we offer a minimum one-year warranty on all parts. Some parts may have extended warranties provided by the manufacturer. Please check the specific part’s warranty details for more information.
---
Do you sell used or refurbished parts?
We primarily sell brand new OEM and aftermarket parts. However, we do offer a selection of refurbished parts which are clearly marked and come with their own warranties. All refurbished parts undergo strict quality control checks.
---
What is the return policy?
You can return any part within 30 days of purchase for a full refund. The part must be in its original packaging and condition. Customized or special-order parts are non-returnable.


In [17]:
SYSTEM_PROMPT = """
You're a senior customer support agent for an online autoparts store.
You're always helpful and answer customer questions only based on the provided
information. If you don't know the answer - just reply with an excuse that you
don't know. Keep your answers brief and to the point. Be kind and respectful.

Use the provided context for your answers. The most relevant information is
at the top. Each piece of information is separated by ---.
"""

In [18]:
def create_user_prompt(query: str) -> str:
    return dedent(
        f"""
Use the following information:

```
{retrieve_context(query)}
```

to answer the question:
{query}
    """
    )

In [19]:
print(create_user_prompt(query))


Use the following information:

```
Do you offer a warranty on your parts?
Yes, we offer a minimum one-year warranty on all parts. Some parts may have extended warranties provided by the manufacturer. Please check the specific part’s warranty details for more information.
---
Do you sell used or refurbished parts?
We primarily sell brand new OEM and aftermarket parts. However, we do offer a selection of refurbished parts which are clearly marked and come with their own warranties. All refurbished parts undergo strict quality control checks.
---
What is the return policy?
You can return any part within 30 days of purchase for a full refund. The part must be in its original packaging and condition. Customized or special-order parts are non-returnable.
```

to answer the question:
What is the warranty for the parts?



### Ask Questions

In [20]:
user_prompt = create_user_prompt(query)

messages = [
    {
        "role": "system",
        "content": SYSTEM_PROMPT,
    },
]

In [21]:
def call_model(query: str, messages: List) -> ChatCompletionMessage:
    messages.append(
        {
            "role": "user",
            "content": create_user_prompt(query),
        },
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
    )
    message = response.choices[0].message
    messages.append(message)
    return message

In [22]:
query = "What is the warranty for the parts?"
message = call_model(query, messages)
print(message)

ChatCompletionMessage(content='We offer a minimum one-year warranty on all parts. Some parts may have extended warranties provided by the manufacturer.', role='assistant', function_call=None, tool_calls=None)


In [23]:
print(message.content)

We offer a minimum one-year warranty on all parts. Some parts may have extended warranties provided by the manufacturer.


In [24]:
query = "Can I return a part that I incorrectly ordered?"
message = call_model(query, messages)
print(message.content)

Yes, you can return a part within 30 days of purchase for a full refund, as long as it's in its original packaging and condition.


In [25]:
query = "How much does an OEM water pump cost for W211?"
message = call_model(query, messages)
print(message.content)

I don't know. The provided information doesn't mention specific prices for parts, including the OEM water pump for W211.
